# agent with Llamaindex


# Agent with LlamaIndex Notebook

This notebook demonstrates the implementation of a simple agent using LlamaIndex framework. It showcases:

- Setting up a local LLM model using Ollama
- Creating custom functions for multiplication and weather data retrieval
- Implementing an agent workflow with function tools
- Demonstrating both stateless and stateful agent interactions
- Using context objects to maintain conversation history

The agent is able to perform mathematical operations and fetch weather information through defined function tools.


In [63]:
pip install llama-index llama-index-llms-ollama

You should consider upgrading via the '/Users/yohannesyimam/Documents/Projects/deffusion_model/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [64]:
pip install --upgrade typing_extensions # use python version >= 3.10 or update type extensions

You should consider upgrading via the '/Users/yohannesyimam/Documents/Projects/deffusion_model/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [65]:
from llama_index.core.agent.workflow import (AgentWorkflow,
                                             FunctionAgent,
                                             ReActAgent)
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama

# initialize local model

In [66]:
llm = Ollama(model="dagbs/qwen2.5-coder-7b-instruct-abliterated:latest")

## functions

In [67]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

def multiply(a:int, b:int) -> int:
    """Multiply two numbers and return the result."""
    return a * b

In [68]:
import requests

def get_weather(city: str) -> str:
    """ a function that returns the weather data as a string value"""
    url = "https://api.weatherapi.com/v1/current.json"
    params = {"key": "e40acd9bb9154ea499d34832252505", "q": city, "aqi":"no"}

    resp = requests.get(url, params=params)
    if resp.status_code == 200:
        response_data = resp.json()
        return response_data["current"]["condition"]["text"]
    else:
        raise Exception("not data found")
    # return f"The weather in {city} is sunny today."

In [69]:
# agent configurations

calculator_agent = ReActAgent(
    name = "Calculator",
    description="Performs basic arithmetic operations",
    system_prompt="you are a calculator assistant, Use your tools for any math operations",
    tools=[add, subtract, multiply],
    llm=llm
)

weather_agent = ReActAgent(
    name="weather",
    description="Useful for getting the weather for a given location.",
    system_prompt="you are a weather assistant, Use your tools for any weather information",
    tools=[get_weather],
    llm=llm
)


## agent

In [70]:
agent = AgentWorkflow(agents=[calculator_agent, weather_agent], root_agent="weather")

Agents are stateless by default, add remembering past interactions is opt-in using a Context object This might be useful if you want to use an agent that needs to remember previous interactions, like a chatbot that maintains context across multiple messages or a task manager that needs to track progress over time.

In [ ]:
# stateless
response = await agent.run(user_msg="What is 2 times 3?")
print(response)
# remembering state
from llama_index.core.workflow import Context

ctx = Context(agent)

await agent.run(user_msg="my name is john", ctx=ctx)
response = await agent.run(user_msg="What was my name ", ctx=ctx)
print(response)


In [75]:
print(await agent.run(user_msg="what is my name?", ctx=ctx))

Your name is John.


In [74]:
print(await agent.run(user_msg="what is the weather in new york city?"))

The weather in New York City is overcast.
